In [69]:
import os
import numpy as np
import torch

import experiment
import utils
import clip_utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
output_path = experiment.get_current_path()
output_path

'/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08'

In [32]:
path = os.path.join(output_path, "instance_features")

In [83]:
files = utils.get_all_files_in_dir_and_subdir(path, "classes.pl")
files

['/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/test/office4_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/test/room2_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/train/office0_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/train/office1_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/train/office2_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/train/room0_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/instance_features/val/office3_predicted_classes.pl',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-

In [84]:
gt_labels = "dataset/data/replica_split/ground_truth"
mask_paths = "experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d"

In [85]:
total = 0.0
correct = 0.0

for file in files:
    name = os.path.basename(file).split("_")[0]
    print(name)
    split = os.path.basename(os.path.dirname(file))
    print(split)
    gt = np.load(os.path.join(gt_labels, f"{name}.npy"))
    print(gt.shape)
    masks = torch.load(os.path.join(mask_paths, split, f"{name}_masks.pt"))
    print("Mask 3D instance masks: ", len(masks))
    pred_classes = torch.load(file)
    print(pred_classes.shape)
    
    for i, mask in enumerate(masks):
        mask = mask != 0
        gt_instance_label = int(gt[mask].mean())
        if gt_instance_label == -1:
            continue
        #print(gt_instance_label, pred_classes[i])
        if gt_instance_label == pred_classes[i]:
            correct += 1.0
        total += 1.0
print("Accuracy: ", correct / total)

office4
test
(993008,)
Mask 3D instance masks:  74
torch.Size([74])
room2
test
(722496,)
Mask 3D instance masks:  80
torch.Size([80])
office0
train
(589517,)
Mask 3D instance masks:  76
torch.Size([76])
office1
train
(423007,)
Mask 3D instance masks:  61
torch.Size([61])
office2
train
(858623,)
Mask 3D instance masks:  69
torch.Size([69])
room0
train
(954492,)
Mask 3D instance masks:  73
torch.Size([73])
office3
val
(1187140,)
Mask 3D instance masks:  63
torch.Size([63])
room1
val
(645512,)
Mask 3D instance masks:  70
torch.Size([70])
Accuracy:  0.038181818181818185


## Openscene:

In [59]:
instance_path = os.path.join(output_path, "openscene")
npy_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(instance_path)
    for file in files
    if file.endswith(".npy")
]
npy_files

['/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/test/office4_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/test/room2_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/val/room1_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/val/office3_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/train/office1_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/train/room0_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/train/office0_features.npy',
 '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/train/office2_features.npy']

In [60]:
from clip_utils import extract_text_feature, REPLICA_LABELS#, MATTERPORT_LABELS_21

In [61]:
labelset = list(REPLICA_LABELS)
text_features, new_label_set = extract_text_feature(labelset)

Use prompt engineering: a XX in a scene
Loading CLIP ViT-B/32 model...
Finish loading


In [79]:
labelset

['basket',
 'bed',
 'bench',
 'bin',
 'blanket',
 'blinds',
 'book',
 'bottle',
 'box',
 'bowl',
 'camera',
 'cabinet',
 'candle',
 'chair',
 'clock',
 'cloth',
 'comforter',
 'cushion',
 'desk',
 'desk-organizer',
 'door',
 'indoor-plant',
 'lamp',
 'monitor',
 'nightstand',
 'panel',
 'picture',
 'pillar',
 'pillow',
 'pipe',
 'plant-stand',
 'plate',
 'pot',
 'sculpture',
 'shelf',
 'sofa',
 'stool',
 'switch',
 'table',
 'tablet',
 'tissue-paper',
 'tv-screen',
 'tv-stand',
 'vase',
 'vent',
 'wall-plug',
 'window',
 'rug']

In [81]:
total = 0.0
correct = 0.0

for file in npy_files:
    instance_features = np.load(file)
    instance_features = torch.Tensor(instance_features)
    
    sample_name = os.path.basename(file).split('_')[0]
    
    print(f"\nProcessing {sample_name}")
    
    print(text_features.shape)
    print(instance_features.shape)

    gt = np.load(os.path.join(gt_labels, f"{sample_name}.npy")) 
    print(gt.shape) 
    predicted_classes, confidence_scores = clip_utils.classify_features(text_features, instance_features)
    print("Predicted_classes: ", predicted_classes.shape)
    for i,point in enumerate(gt):
        if point == -1:
            continue
        total += 1.0
        if point == predicted_classes[i]:
            correct += 1.0
            
print("Accuracy: ", correct/total)


Processing office4
torch.Size([48, 512])
torch.Size([993008, 512])
(993008,)
Predicted_classes:  torch.Size([993008])

Processing room2
torch.Size([48, 512])
torch.Size([722496, 512])
(722496,)
Predicted_classes:  torch.Size([722496])

Processing room1
torch.Size([48, 512])
torch.Size([645512, 512])
(645512,)
Predicted_classes:  torch.Size([645512])

Processing office3
torch.Size([48, 512])
torch.Size([1187140, 512])
(1187140,)
Predicted_classes:  torch.Size([1187140])

Processing office1
torch.Size([48, 512])
torch.Size([423007, 512])
(423007,)
Predicted_classes:  torch.Size([423007])

Processing room0
torch.Size([48, 512])
torch.Size([954492, 512])
(954492,)
Predicted_classes:  torch.Size([954492])

Processing office0
torch.Size([48, 512])
torch.Size([589517, 512])
(589517,)
Predicted_classes:  torch.Size([589517])

Processing office2
torch.Size([48, 512])
torch.Size([858623, 512])
(858623,)
Predicted_classes:  torch.Size([858623])
Accuracy:  0.434133585684646


In [89]:
MATTERPORT_COLOR_MAP_160 = {
    1: (174., 199., 232.), # wall
    2: (214., 39., 40.), # door
    3: (186., 197., 62.), # ceiling
    4: (152., 223., 138.), # floor
    5: (196., 156., 148.), # picture
    6: (197., 176., 213.), # window
    7: (188., 189., 34.), # chair
    8: (141., 91., 229.), # pillow
    9: (237.0, 204.0, 37.0), # lamp
    10: (31., 119., 180.), # cabinet
    11: (219., 219., 141.), # curtain
    12: (255., 152., 150.), # table
    13: (150.0, 53.0, 56.0), # plant
    14: (162.0, 62.0, 60.0), # mirror
    15: (62.0, 143.0, 148.0), # towel
    16: (112., 128., 144.), # sink
    17: (229.0, 91.0, 104.0), # shelves
    18: (140., 86., 75.), # sofa
    19: (255., 187., 120.), # bed
    20: (137.0, 63.0, 14.0), # night stand
    21: (44., 160., 44.), # toilet
    22: (39.0, 19.0, 208.0), # column
    23: (64.0, 158.0, 70.0), # banister
    24: (208.0, 49.0, 84.0), # stairs
    25: (90.0, 119.0, 201.0), # stool
    26: (118., 174., 76.), # vase
    27: (143.0, 45.0, 115.0), # television
    28: (153., 108., 234.), # pot
    29: (247., 182., 210.), # desk
    30: (177.0, 82.0, 239.0), # box
    31: (58.0, 98.0, 137.0), # coffee table
    32: (23., 190., 207.), # counter
    33: (17.0, 242.0, 171.0), # bench
    34: (79.0, 55.0, 137.0), # garbage bin
    35: (127.0, 63.0, 52.0), # fireplace
    36: (34.0, 14.0, 130.0), # clothes
    37: (227., 119., 194.), # bathtub
    38: (192.0, 229.0, 91.0), # book
    39: (49.0, 206.0, 87.0), # air vent
    40: (250., 253., 26.), # faucet
    41: (0., 0., 0.), # unlabel/unknown
    80: (82., 75., 227.),
    82: (253., 59., 222.),
    84: (240., 130., 89.),
    86: (123., 172., 47.),
    87: (71., 194., 133.),
    88: (24., 94., 205.),
    89: (134., 16., 179.),
    90: (159., 32., 52.),
    93: (213., 208., 88.),
    95: (64., 158., 70.),
    96: (18., 163., 194.),
    97: (65., 29., 153.),
    98: (177., 10., 109.),
    99: (152., 83., 7.),
    100: (83., 175., 30.),
    101: (18., 199., 153.),
    102: (61., 81., 208.),
    103: (213., 85., 216.),
    104: (170., 53., 42.),
    105: (161., 192., 38.),
    106: (23., 241., 91.),
    107: (12., 103., 170.),
    110: (151., 41., 245.),
    112: (133., 51., 80.),
    115: (184., 162., 91.),
    116: (50., 138., 38.),
    118: (31., 237., 236.),
    120: (39., 19., 208.),
    121: (223., 27., 180.),
    122: (254., 141., 85.),
    125: (97., 144., 39.),
    128: (106., 231., 176.),
    130: (12., 61., 162.),
    131: (124., 66., 140.),
    132: (137., 66., 73.),
    134: (250., 253., 26.),
    136: (55., 191., 73.),
    138: (60., 126., 146.),
    139: (153., 108., 234.),
    140: (184., 58., 125.),
    141: (135., 84., 14.),
    145: (139., 248., 91.),
    148: (53., 200., 172.),
    154: (63., 69., 134.),
    155: (190., 75., 186.),
    156: (127., 63., 52.),
    157: (141., 182., 25.),
    159: (56., 144., 89.),
    161: (64., 160., 250.),
    163: (182., 86., 245.),
    165: (139., 18., 53.),
    166: (134., 120., 54.),
    168: (49., 165., 42.),
    169: (51., 128., 133.),
    170: (44., 21., 163.),
    177: (232., 93., 193.),
    180: (176., 102., 54.),
    185: (116., 217., 17.),
    188: (54., 209., 150.),
    191: (60., 99., 204.),
    193: (129., 43., 144.),
    195: (252., 100., 106.),
    202: (187., 196., 73.),
    208: (13., 158., 40.),
    213: (52., 122., 152.),
    214: (128., 76., 202.),
    221: (187., 50., 115.),
    229: (180., 141., 71.),
    230: (77., 208., 35.),
    232: (72., 183., 168.),
    233: (97., 99., 203.),
    242: (172., 22., 158.),
    250: (155., 64., 40.),
    261: (118., 159., 30.),
    264: (69., 252., 148.),
    276: (45., 103., 173.),
    283: (111., 38., 149.),
    286: (184., 9., 49.),
    300: (188., 174., 67.),
    304: (53., 206., 53.),
    312: (97., 235., 252.),
    323: (66., 32., 182.),
    325: (236., 114., 195.),
    331: (241., 154., 83.),
    342: (133., 240., 52.),
    356: (16., 205., 144.),
    370: (75., 101., 198.),
    392: (237., 95., 251.),
    395: (191., 52., 49.),
    399: (227., 254., 54.),
    408: (49., 206., 87.),
    417: (48., 113., 150.),
    488: (125., 73., 182.),
    540: (229., 32., 114.),
    562: (158., 119., 28.),
    570: (60., 205., 27.),
    572: (18., 215., 201.),
    581: (79., 76., 153.),
    609: (134., 13., 116.),
    748: (192., 97., 63.),
    776: (108., 163., 18.),
    1156: (95., 220., 156.),
    1163: (98., 141., 208.),
    1164: (144., 19., 193.),
    1165: (166., 36., 57.),
    1166: (212., 202., 34.),
    1167: (23., 206., 34.),
    1168: (91., 211., 236.),
    1169: (79., 55., 137.),
    1170: (182., 19., 117.),
    1171: (134., 76., 14.),
    1172: (87., 185., 28.),
    1173: (82., 224., 187.),
    1174: (92., 110., 214.),
    1175: (168., 80., 171.),
    1176: (197., 63., 51.),
    1178: (175., 199., 77.),
    1179: (62., 180., 98.),
    1180: (8., 91., 150.),
    1181: (77., 15., 130.),
    1182: (154., 65., 96.),
    1183: (197., 152., 11.),
    1184: (59., 155., 45.),
    1185: (12., 147., 145.),
    1186: (54., 35., 219.),
    1187: (210., 73., 181.),
    1188: (221., 124., 77.),
    1189: (149., 214., 66.),
    1190: (72., 185., 134.),
    1191: (42., 94., 198.),
    1200: (0, 0, 0)
}

In [100]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
def visualize_labels(u_index, labels, palette, out_name, loc='lower left', ncol=7):
    patches = []
    for i, index in enumerate(u_index):
        label = labels[index]
        cur_color = [palette[index * 3] / 255.0, palette[index * 3 + 1] / 255.0, palette[index * 3 + 2] / 255.0]
        red_patch = mpatches.Patch(color=cur_color, label=label)
        patches.append(red_patch)
    plt.figure()
    plt.axis('off')
    legend = plt.legend(frameon=False, handles=patches, loc=loc, ncol=ncol, bbox_to_anchor=(0, -0.3), prop={'size': 5}, handlelength=0.7)
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent()
    bbox = bbox.from_extents(*(bbox.extents + np.array([-5,-5,5,5])))
    bbox = bbox.transformed(fig.dpi_scale_trans.inverted())
    plt.savefig(out_name, bbox_inches=bbox, dpi=300)
    plt.close()
    
def get_palette(num_cls=21, colormap='scannet'):
    if colormap == 'scannet':
        scannet_palette = []
        for _, value in SCANNET_COLOR_MAP_20.items():
            scannet_palette.append(np.array(value))
        palette = np.concatenate(scannet_palette)
    elif colormap == 'matterport':
        scannet_palette = []
        for _, value in MATTERPORT_COLOR_MAP_21.items():
            scannet_palette.append(np.array(value))
        palette = np.concatenate(scannet_palette)
    elif colormap == 'matterport_160':
        scannet_palette = []
        for _, value in MATTERPORT_COLOR_MAP_160.items():
            scannet_palette.append(np.array(value))
        palette = np.concatenate(scannet_palette)
    elif colormap == 'nuscenes16':
        nuscenes16_palette = []
        for _, value in NUSCENES16_COLORMAP.items():
            nuscenes16_palette.append(np.array(value))
        palette = np.concatenate(nuscenes16_palette)
    else:
        n = num_cls
        palette = [0]*(n*3)
        for j in range(0,n):
            lab = j
            palette[j*3+0] = 0
            palette[j*3+1] = 0
            palette[j*3+2] = 0
            i = 0
            while lab > 0:
                palette[j*3+0] |= (((lab >> 0) & 1) << (7-i))
                palette[j*3+1] |= (((lab >> 1) & 1) << (7-i))
                palette[j*3+2] |= (((lab >> 2) & 1) << (7-i))
                i = i + 1
                lab >>= 3
    return palette

In [126]:
palette = get_palette(colormap='matterport_160')
print(palette)

[174. 199. 232. 214.  39.  40. 186. 197.  62. 152. 223. 138. 196. 156.
 148. 197. 176. 213. 188. 189.  34. 141.  91. 229. 237. 204.  37.  31.
 119. 180. 219. 219. 141. 255. 152. 150. 150.  53.  56. 162.  62.  60.
  62. 143. 148. 112. 128. 144. 229.  91. 104. 140.  86.  75. 255. 187.
 120. 137.  63.  14.  44. 160.  44.  39.  19. 208.  64. 158.  70. 208.
  49.  84.  90. 119. 201. 118. 174.  76. 143.  45. 115. 153. 108. 234.
 247. 182. 210. 177.  82. 239.  58.  98. 137.  23. 190. 207.  17. 242.
 171.  79.  55. 137. 127.  63.  52.  34.  14. 130. 227. 119. 194. 192.
 229.  91.  49. 206.  87. 250. 253.  26.   0.   0.   0.  82.  75. 227.
 253.  59. 222. 240. 130.  89. 123. 172.  47.  71. 194. 133.  24.  94.
 205. 134.  16. 179. 159.  32.  52. 213. 208.  88.  64. 158.  70.  18.
 163. 194.  65.  29. 153. 177.  10. 109. 152.  83.   7.  83. 175.  30.
  18. 199. 153.  61.  81. 208. 213.  85. 216. 170.  53.  42. 161. 192.
  38.  23. 241.  91.  12. 103. 170. 151.  41. 245. 133.  51.  80. 184.
 162. 

In [98]:
test_path = os.path.join(output_path, "test")
os.makedirs(test_path, exist_ok=True)

In [102]:
visualize_labels(list(range(len(labelset))),labelset,palette, os.path.join(test_path, "labels.png"), ncol=5)

In [146]:
mask3d_path = os.path.join(output_path, "mask3d")
#split = 'val'
mask_paths = utils.get_all_files_in_dir_and_subdir(mask3d_path, "pt")#sorted(glob(os.path.join(mask3d_path, split, '*.pt')))

openscene_path = os.path.join(output_path, "openscene")
#features_paths = sorted(glob(os.path.join(openscene_path, '*.npy')))
point_cloud_paths = utils.get_all_files_in_dir_and_subdir(openscene_path, "input.ply")

features_path = utils.get_all_files_in_dir_and_subdir(openscene_path, "npy")

pred_classes = utils.get_all_files_in_dir_and_subdir(os.path.join(output_path, "instance_features"), "classes.pl")

per_point_gt_paths = "dataset/data/replica_split/ground_truth"

print(mask_paths)
#print(features_paths)
print(point_cloud_paths)
print(features_path)
print(per_point_gt_paths)

['/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/test/office4_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/test/room2_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/train/office0_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/train/office1_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/train/office2_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/train/room0_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/val/office3_masks.pt', '/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/mask3d/val/room1_masks.pt']
['/cluster/54/blessman/ml3d/experiments/merged_pipline/run_2025-01-29-20-44-08/openscene/test/office4_input.ply', '/cluster/54/bles

In [138]:
import point_cloud
import open3d as o3d
%matplotlib inline 

ply_data = o3d.io.read_point_cloud(point_cloud_paths[0])

name = os.path.basename(point_cloud_paths[0]).split("_")[0]
print(name)

# Extract points and colors
coords = np.asarray(ply_data.points)  # 3D coordinates
colors = np.asarray(ply_data.colors)  # RGB values (normalized to [0, 1])

print(colors.shape)

# Set base color
colors[:] = 0.5

# Normalize colors to 0-255 and convert to hexadecimal
colors = (colors * 255).astype(np.uint64)

gt = np.load(os.path.join(gt_labels, f"{name}.npy"))
print(gt.shape)

for i,point in enumerate(gt):
    if point == -1:
        continue
    colors[i] = (palette[point*3],palette[point*3+1],palette[point*3+2])

#for i, mask in enumerate(binary_masks):
#    random_index = random.randint(1, len(MATTERPORT_COLOR_MAP_21)-1)
 #   colors[mask] = MATTERPORT_COLOR_MAP_21[random_index]

#colors_hex = (colors[:, 0] << 16) + (colors[:, 1] << 8) + colors[:, 2]

# Visualize with k3d
point_cloud.visualize_point_cloud_with_k3d(coords, colors,point_size=2, is_norm=True, is_rgb=True).display()



# Extract points and colors
coords = np.asarray(ply_data.points)  # 3D coordinates
colors = np.asarray(ply_data.colors)  # RGB values (normalized to [0, 1])

print(colors.shape)

# Set base color
colors[:] = 0.5

# Normalize colors to 0-255 and convert to hexadecimal
colors = (colors * 255).astype(np.uint64)

pred_cl = torch.load(pred_classes[0])
print(pred_cl.shape)
masks = torch.load(mask_paths[0])

for i,mask in enumerate(masks):
    if point == -1:
        continue
    colors[mask] = (palette[pred_cl[i]*3],palette[pred_cl[i]*3+1],palette[pred_cl[i]*3+2])

#for i, mask in enumerate(binary_masks):
#    random_index = random.randint(1, len(MATTERPORT_COLOR_MAP_21)-1)
 #   colors[mask] = MATTERPORT_COLOR_MAP_21[random_index]

#colors_hex = (colors[:, 0] << 16) + (colors[:, 1] << 8) + colors[:, 2]

# Visualize with k3d
point_cloud.visualize_point_cloud_with_k3d(coords, colors,point_size=2, is_norm=True, is_rgb=True).display()



office4
(993008, 3)
(993008,)


Output()

(993008, 3)
torch.Size([74])


Output()

In [147]:
instance_features = torch.from_numpy(np.load(features_path[0]))
print(instance_features.shape)
predicted_classes, confidence_scores = clip_utils.classify_features(text_features, instance_features)
print(predicted_classes.shape)

ply_data = o3d.io.read_point_cloud(point_cloud_paths[0])

name = os.path.basename(point_cloud_paths[0]).split("_")[0]
print(name)

# Extract points and colors
coords = np.asarray(ply_data.points)  # 3D coordinates
colors = np.asarray(ply_data.colors)  # RGB values (normalized to [0, 1])

print(colors.shape)

# Set base color
colors[:] = 0.5

# Normalize colors to 0-255 and convert to hexadecimal
colors = (colors * 255).astype(np.uint64)

for i,point in enumerate(predicted_classes):
    colors[i] = (palette[point*3],palette[point*3+1],palette[point*3+2])

#for i, mask in enumerate(binary_masks):
#    random_index = random.randint(1, len(MATTERPORT_COLOR_MAP_21)-1)
 #   colors[mask] = MATTERPORT_COLOR_MAP_21[random_index]

#colors_hex = (colors[:, 0] << 16) + (colors[:, 1] << 8) + colors[:, 2]

# Visualize with k3d
point_cloud.visualize_point_cloud_with_k3d(coords, colors,point_size=2, is_norm=True, is_rgb=True).display()

torch.Size([993008, 512])
torch.Size([993008])
office4
(993008, 3)


Output()

In [152]:
# Extract points and colors
coords = np.asarray(ply_data.points)  # 3D coordinates
colors = np.asarray(ply_data.colors)  # RGB values (normalized to [0, 1])

print(colors.shape)

# Set base color
colors[:] = 0.5

# Normalize colors to 0-255 and convert to hexadecimal
colors = (colors * 255).astype(np.uint64)

gt = np.load(os.path.join(gt_labels, f"{name}.npy"))
print(gt.shape)
print(pred_cl.shape)
masks = torch.load(mask_paths[0])

for i,mask in enumerate(masks):
    instance_id = int(gt[mask].mean().item())
    if instance_id == -1:
        continue
    colors[mask] = (palette[instance_id*3],palette[instance_id*3+1],palette[instance_id*3+2])

#for i, mask in enumerate(binary_masks):
#    random_index = random.randint(1, len(MATTERPORT_COLOR_MAP_21)-1)
 #   colors[mask] = MATTERPORT_COLOR_MAP_21[random_index]

#colors_hex = (colors[:, 0] << 16) + (colors[:, 1] << 8) + colors[:, 2]

# Visualize with k3d
point_cloud.visualize_point_cloud_with_k3d(coords, colors,point_size=2, is_norm=True, is_rgb=True).display()

(993008, 3)
(993008,)
torch.Size([74])


Output()